In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from py4DSTEM.io import read
import py4DSTEM
import tifffile as tf
# import torch


In [ ]:
offset_heatmap = np.load('/Users/cadenmyers/billingelab/dev/skyrmion_lattices/yevgeny_proj/data/offsetheatmap_4fold.npz')['data']
datacube = read('/Users/cadenmyers/billingelab/dev/skyrmion_lattices/yevgeny_proj/data/0020 - original-centralized-masked.h5')



In [ ]:
def normalize_min_max(data):
    if isinstance(data, torch.Tensor):
        array = data.detach().numpy()
    else:
        array = data
    array_min = np.min(array)
    array_max = np.max(array)
    norm_array = (array - array_min) / (array_max - array_min)
    if isinstance(data, torch.Tensor):
        norm_tensor = torch.tensor(norm_array)
        return norm_tensor
    else:
        return norm_array

In [ ]:
threshold_angle = 30
coords = np.where(offset_heatmap >= threshold_angle) # coords where offset value is greater than threshold
x_coord, y_coord = coords

# Get DP and x, y from datacube
dplist = []
for i in range(x_coord.shape[0]):
    rx = x_coord[i]
    ry = y_coord[i]
    # print(rx, ry)
    dp = datacube.data[rx][ry]
    dp_and_coord = {'dp': dp, 'xcoord': rx, 'ycoord': ry}
    dplist.append(dp_and_coord)

# Sum all the DPs
sum_dp = np.zeros(dplist[0]['dp'].shape) # empty array of same shape as dp's
for dict in dplist:
    dp = dict['dp']
    dp_norm = normalize_min_max(dp)
    sum_dp += dp_norm

plt.imshow(sum_dp)
plt.title(f'Total DPs summed: {len(dplist)}, angle > {threshold_angle}')
plt.colorbar()

In [ ]:
threshold_angle = 30
x_coord = np.where((offset_heatmap <= threshold_angle) & (offset_heatmap >= -threshold_angle))[0] # X-coord where offset value is greater than threshold
y_coord = np.where((offset_heatmap <= threshold_angle) & (offset_heatmap >= -threshold_angle))[1] # Y-coord where offset value is greater than threshold

# Get DP and x, y from datacube
dplist = []
for i in range(x_coord.shape[0]):
    rx = x_coord[i]
    ry = y_coord[i]
    # print(rx, ry)
    dp = datacube.data[rx][ry]
    dp_and_coord = {'dp': dp, 'xcoord': rx, 'ycoord': ry}
    dplist.append(dp_and_coord)

# Sum all the DPs
sum_dp = np.zeros(dplist[0]['dp'].shape) # empty array of same shape as dp's
for dict in dplist:
    dp = dict['dp']
    dp_norm = normalize_min_max(dp)
    sum_dp += dp_norm

plt.imshow(sum_dp)
plt.title(f'Total DPs summed: {len(dplist)}, angle < {threshold_angle}')
plt.colorbar()

In [ ]:
# CREATE DATACUBE OF DPs GREATER THAN 30

threshold_angle = 30

# Get coordinates of interest
coords = np.where(offset_heatmap >= threshold_angle)
x_coord, y_coord = coords

original_shape = datacube.data.shape  # (142, 217, 256, 256)

# Initialize new datacube array with zeros
new_data = np.zeros_like(datacube.data)

# Populate new datacube with DPs from the original datacube
for rx, ry in zip(x_coord, y_coord):
    new_data[rx, ry] = datacube.data[rx, ry].copy()

# Wrap in a py4DSTEM DataCube
dc_pos30 = py4DSTEM.DataCube(new_data)

# Perform operations on the new DataCube
mean_dp_pos30 = dc_pos30.get_dp_mean()  # Mean diffraction pattern
max_dp_pos30 = dc_pos30.get_dp_max() 


In [ ]:
mean_dp_array = mean_dp.data
max_dp_array = max_dp.data

# plt.imshow(max_dp_array)
plt.imshow(mean_dp_array)
plt.colorbar()

In [ ]:
# CREATE DATACUBE OF DPs GREATER THAN 30

threshold_angle = -30

# Get coordinates of interest
coords = np.where(offset_heatmap <= threshold_angle)
x_coord, y_coord = coords

original_shape = datacube.data.shape  # (142, 217, 256, 256)

# Initialize new datacube array with zeros
new_data = np.zeros_like(datacube.data)

# Populate new datacube with DPs from the original datacube
for rx, ry in zip(x_coord, y_coord):
    new_data[rx, ry] = datacube.data[rx, ry].copy()

# Wrap in a py4DSTEM DataCube
dc_neg30 = py4DSTEM.DataCube(new_data)

# Perform operations on the new DataCube
mean_dp_neg30 = dc_neg30.get_dp_mean()
max_dp_neg30 = dc_neg30.get_dp_max() 


In [ ]:
mean_dp_array = mean_dp_neg30.data
max_dp_array = max_dp_neg30.data

plt.imshow(mean_dp_array)
plt.colorbar()

In [ ]:
# ALL DPs BETWEEN -30 AND 30 DEGREES

threshold_angle = 30

# Get coordinates of interest
coords = np.where((offset_heatmap < threshold_angle) & (offset_heatmap > -threshold_angle))
x_coord, y_coord = coords

original_shape = datacube.data.shape  # (142, 217, 256, 256)

# Initialize new datacube array with zeros
new_data = np.zeros_like(datacube.data)

# Populate new datacube with DPs from the original datacube
for rx, ry in zip(x_coord, y_coord):
    dp = datacube.data[rx, ry]
    if dp.sum() > 0: # remove empty arrays
        new_data[rx, ry] = datacube.data[rx, ry].copy()

# Wrap in a py4DSTEM DataCube
dc_else = py4DSTEM.DataCube(new_data)

# Perform operations on the new DataCube
mean_dp_else = dc_else.get_dp_mean()
max_dp_else = dc_else.get_dp_max() 

In [ ]:
print(dc_else.data.shape)

mean_dp_else_array = mean_dp_else.data
max_dp_array = max_dp_else.data
# plt.imshow(mean_dp_else_array)
# plt.imshow(max_dp_array)

# plt.imshow(dc_else.data[10][10])
plt.colorbar()
